In [ ]:
import graphlab as gl
import pandas as pd
import numpy as np
import scipy.stats.stats as stats
#%load_ext rpy2.ipython
import itertools
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics, preprocessing, datasets   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Performing grid search
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import LabelKFold
from sklearn.cross_validation import LeaveOneLabelOut
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 8, 8
import sys
import pickle
from graphlab.util import cloudpickle
import graphlab.aggregate as agg
from math import sqrt

In [ ]:
# Load the data
data =  gl.SFrame.read_csv('/net/fowler/vol1/home/vegray/metaDMS/dato/scaled/data/dmsTraining_2017-02-20.csv')

In [ ]:
## Use predictors3
predictors3 = ['aa1', 'aa2', 'wt_mut', 'aa1_polarity', 
               'aa2_polarity', 'aa1_PI', 'aa2_PI', 'deltaPI', 
               'Grantham', 'aa1_weight', 'aa2_weight', 'deltaWeight',
               'aa1vol', 'aa2vol', 'deltavolume', 'aa1_psic',
               'aa2_psic', 'delta_psic',  'accessibility',
               'dssp_sec_str', 'phi_psi_reg', 'delta_solvent_accessibility',
               'b_factor', 'mut_msa_congruency', 'mut_mut_msa_congruency', 
               'seq_ind_closest_mut',  'evolutionary_coupling_avg']

In [ ]:
## Global model dataset cleanup
data2 = data.filter_by([x for x in data['dms_id'].unique() if x not in ['CcdB','beta-lactamase_2500', 'beta-lactamase_0', 
                                                                                      'kka2_1:1','kka2_1:4','kka2_1:8',
                                                                     'beta-lactamase_39', 'gal4','hemagglutinin', 'np',
                                                                                      'dbr1', 'beta-lactamase_156',
                                                                                      'beta-lactamase_625', 'ERK2','GFP',
                                                                                      'Brca1_HDR', ]], 'dms_id')
#data2 = data2[(data2['binary_mut_type_prob'] < 0.4) | (data2['binary_mut_type_prob'] > 0.6)]
data2 = data2.filter_by([x for x in data2['mut_type'].unique() if x in ['missense']], 'mut_type')
#del data2['id']
#data2 = data2.add_row_number('id',0)

In [ ]:
data2['dms_id'].unique()

In [ ]:
## Make data set w/o structural info

data_str = data2

data_str['accessibility'] = None
data_str['dssp_sec_str'] = None
data_str['dssp_sec_str'] = data_str['dssp_sec_str'].astype(str)
data_str['phi_psi_reg'] = None
data_str['phi_psi_reg'] = data_str['phi_psi_reg'].astype(str)
data_str['delta_solvent_accessibility'] = None
data_str['b_factor'] = None

data_str['mut_mut_msa_congruency'] = data_str['mut_mut_msa_congruency'].astype(str)

In [ ]:
## Make data set w/o structural info

data_bfact = data2

data_bfact['accessibility'] = None
data_bfact['dssp_sec_str'] = None
data_bfact['dssp_sec_str'] = data_bfact['dssp_sec_str'].astype(str)
data_bfact['phi_psi_reg'] = None
data_bfact['phi_psi_reg'] = data_bfact['phi_psi_reg'].astype(str)
data_bfact['delta_solvent_accessibility'] = None
data_bfact['b_factor'] = None

data_bfact['mut_mut_msa_congruency'] = data_bfact['mut_mut_msa_congruency'].astype(str)

In [5]:
## Make data set w/o evolutionary info

data_evol = data2

data_evol['aa1_psic'] = None
data_evol['aa2_psic'] = None
data_evol['delta_psic'] = None
data_evol['mut_msa_congruency'] = None
data_evol['mut_mut_msa_congruency'] = None
data_evol['seq_ind_closest_mut'] = None
data_evol['evolutionary_coupling_avg'] = None

data_evol['mut_mut_msa_congruency'] = data_evol['mut_mut_msa_congruency'].astype(str)


## KKA2

In [6]:
unpickler = gl._gl_pickle.GLUnpickler(filename = '/net/fowler/vol1/home/vegray/metaDMS/dato/scaled/models/LOPO_models/NoKka2_model_regressionModel_noSELCO_limited_2017-02-08')
NoKka2_model = unpickler.load()
unpickler.close()

In [ ]:
## Missing structural features
Kka2_str = data_str.filter_by([x for x in data_str['dms_id'].unique() if x in ['kka2_1:2']], 'dms_id')
Kka2_str = Kka2_str.filter_by([x for x in Kka2_str['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del Kka2_str['id']
Kka2_str = Kka2_str.add_row_number('id',0)

pred = NoKka2_model.predict(Kka2_str)

stats.pearsonr(pred, Kka2_str['scaled_effect1'])
stats.spearmanr(pred, Kka2_str['scaled_effect1'])

In [10]:
## Missing evolutionary features
Kka2_evol = data_evol.filter_by([x for x in data_evol['dms_id'].unique() if x in ['kka2_1:2']], 'dms_id')
Kka2_evol = Kka2_evol.filter_by([x for x in Kka2_evol['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del Kka2_evol['id']
Kka2_evol = Kka2_evol.add_row_number('id',0)

pred = NoKka2_model.predict(Kka2_evol)


stats.pearsonr(pred, Kka2_evol['scaled_effect1'])
stats.spearmanr(pred, Kka2_evol['scaled_effect1'])

SpearmanrResult(correlation=0.49501403312345593, pvalue=0.0)

In [ ]:
## Missing bfact feature
Kka2_bfact = data_bfact.filter_by([x for x in data_bfact['dms_id'].unique() if x in ['kka2_1:2']], 'dms_id')
Kka2_bfact = Kka2_bfact.filter_by([x for x in Kka2_bfact['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del Kka2_bfact['id']
Kka2_bfact = Kka2_bfact.add_row_number('id',0)

pred = NoKka2_model.predict(Kka2_bfact)


stats.pearsonr(pred, Kka2_bfact['scaled_effect1'])
stats.spearmanr(pred, Kka2_bfact['scaled_effect1'])

## PDZ domain

In [8]:
unpickler = gl._gl_pickle.GLUnpickler(filename = '/net/fowler/vol1/home/vegray/metaDMS/dato/scaled/models/LOPO_models/NoPSD95pdz3_model_regressionModel_noSELCO_limited_2017-02-08')
NoPSD95pdz3_model = unpickler.load()
unpickler.close()

In [ ]:
## Missing structural features
PSD95pdz3_str = data_str.filter_by([x for x in data_str['dms_id'].unique() if x in ['PSD95pdz3']], 'dms_id')
PSD95pdz3_str = PSD95pdz3_str.filter_by([x for x in PSD95pdz3_str['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del PSD95pdz3_str['id']
PSD95pdz3_str = PSD95pdz3_str.add_row_number('id',0)

pred = NoPSD95pdz3_model.predict(PSD95pdz3_str)

stats.pearsonr(pred, PSD95pdz3_str['scaled_effect1'])
stats.spearmanr(pred, PSD95pdz3_str['scaled_effect1'])

In [11]:
## Missing evolutionary features
PSD95pdz3_evol = data_evol.filter_by([x for x in data_evol['dms_id'].unique() if x in ['PSD95pdz3']], 'dms_id')
PSD95pdz3_evol = PSD95pdz3_evol.filter_by([x for x in PSD95pdz3_evol['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del PSD95pdz3_evol['id']
PSD95pdz3_evol = PSD95pdz3_evol.add_row_number('id',0)

pred = NoPSD95pdz3_model.predict(PSD95pdz3_evol)
stats.pearsonr(pred, PSD95pdz3_evol['scaled_effect1'])

stats.pearsonr(pred, PSD95pdz3_evol['scaled_effect1'])
stats.spearmanr(pred, PSD95pdz3_evol['scaled_effect1'])

SpearmanrResult(correlation=0.46863990702842956, pvalue=6.6225104489696682e-87)

## Pab1

In [12]:
unpickler = gl._gl_pickle.GLUnpickler(filename = '/net/fowler/vol1/home/vegray/metaDMS/dato/scaled/models/LOPO_models/NoPab1_model_regressionModel_noSELCO_limited_2017-02-08')
NoPab1_model = unpickler.load()
unpickler.close()

In [ ]:
## Missing structural features
pab1_str = data_str.filter_by([x for x in data_str['dms_id'].unique() if x in ['Pab1']], 'dms_id')
pab1_str = pab1_str.filter_by([x for x in pab1_str['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del pab1_str['id']
pab1_str = pab1_str.add_row_number('id',0)

pred = NoPab1_model.predict(pab1_str)

stats.pearsonr(pred, pab1_str['scaled_effect1'])
#stats.spearmanr(pred, pab1_str['scaled_effect1'])

In [14]:
## Missing evolutionary features
pab1_evol = data_evol.filter_by([x for x in data_evol['dms_id'].unique() if x in ['Pab1']], 'dms_id')
pab1_evol = pab1_evol.filter_by([x for x in pab1_evol['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del pab1_evol['id']
pab1_evol = pab1_evol.add_row_number('id',0)

pred = NoPab1_model.predict(pab1_evol)
stats.pearsonr(pred, pab1_evol['scaled_effect1'])

stats.pearsonr(pred, pab1_evol['scaled_effect1'])
#stats.spearmanr(pred, pab1_evol['scaled_effect1'])

(0.49296129877674449, 9.3547008679135554e-74)

## Beta - lactamase

In [15]:
unpickler = gl._gl_pickle.GLUnpickler(filename = '/net/fowler/vol1/home/vegray/metaDMS/dato/scaled/models/LOPO_models/Nobeta_lact_model_regressionModel_noSELCO_limited_2017-02-08')
Nobeta_lact_model = unpickler.load()
unpickler.close()

In [ ]:
## Missing structural features
beta_lact_str = data_str.filter_by([x for x in data_str['dms_id'].unique() if x in ['beta-lactamase']], 'dms_id')
beta_lact_str = beta_lact_str.filter_by([x for x in beta_lact_str['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del beta_lact_str['id']
beta_lact_str = beta_lact_str.add_row_number('id',0)

pred = Nobeta_lact_model.predict(beta_lact_str)

stats.pearsonr(pred, beta_lact_str['scaled_effect1'])
#stats.spearmanr(pred, beta_lact_str['scaled_effect1'])

In [17]:
## Missing evolutionary features
beta_lact_evol = data_evol.filter_by([x for x in data_evol['dms_id'].unique() if x in ['beta-lactamase']], 'dms_id')
beta_lact_evol = beta_lact_evol.filter_by([x for x in beta_lact_evol['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del beta_lact_evol['id']
beta_lact_evol = beta_lact_evol.add_row_number('id',0)

pred = Nobeta_lact_model.predict(beta_lact_evol)
stats.pearsonr(pred, beta_lact_evol['scaled_effect1'])

stats.pearsonr(pred, beta_lact_evol['scaled_effect1'])
#stats.spearmanr(pred, beta_lact_evol['scaled_effect1'])

(0.60320969082891507, 0.0)

## Ubiquitin

In [18]:
unpickler = gl._gl_pickle.GLUnpickler(filename = '/net/fowler/vol1/home/vegray/metaDMS/dato/scaled/models/LOPO_models/NoUba1_model_regressionModel_noSELCO_limited_2017-02-08')
NoUba1_model = unpickler.load()
unpickler.close()

In [ ]:
## Missing structural features
E1_Ubiquitin_str = data_str.filter_by([x for x in data_str['dms_id'].unique() if x in ['E1_Ubiquitin']], 'dms_id')
E1_Ubiquitin_str = E1_Ubiquitin_str.filter_by([x for x in E1_Ubiquitin_str['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del E1_Ubiquitin_str['id']
E1_Ubiquitin_str = E1_Ubiquitin_str.add_row_number('id',0)

pred = NoUba1_model.predict(E1_Ubiquitin_str)

stats.pearsonr(pred, E1_Ubiquitin_str['scaled_effect1'])
#stats.spearmanr(pred, E1_Ubiquitin_str['scaled_effect1'])

In [22]:
## Missing evolutionary features
E1_Ubiquitin_evol = data_evol.filter_by([x for x in data_evol['dms_id'].unique() if x in ['E1_Ubiquitin']], 'dms_id')
E1_Ubiquitin_evol = E1_Ubiquitin_evol.filter_by([x for x in E1_Ubiquitin_evol['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del E1_Ubiquitin_evol['id']
E1_Ubiquitin_evol = E1_Ubiquitin_evol.add_row_number('id',0)

pred = NoUba1_model.predict(E1_Ubiquitin_evol)
stats.pearsonr(pred, E1_Ubiquitin_evol['scaled_effect1'])

stats.pearsonr(pred, E1_Ubiquitin_evol['scaled_effect1'])
#stats.spearmanr(pred, E1_Ubiquitin_evol['scaled_effect1'])

(0.53198854885273017, 2.6143410920958567e-80)

In [ ]:
## Missing structural features
Ubiquitin_str = data_str.filter_by([x for x in data_str['dms_id'].unique() if x in ['Ubiquitin']], 'dms_id')
Ubiquitin_str = Ubiquitin_str.filter_by([x for x in Ubiquitin_str['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del Ubiquitin_str['id']
Ubiquitin_str = Ubiquitin_str.add_row_number('id',0)

pred = NoUba1_model.predict(Ubiquitin_str)

stats.pearsonr(pred, Ubiquitin_str['scaled_effect1'])
#stats.spearmanr(pred, Ubiquitin_str['scaled_effect1'])

In [23]:
## Missing evolutionary features
Ubiquitin_evol = data_evol.filter_by([x for x in data_evol['dms_id'].unique() if x in ['Ubiquitin']], 'dms_id')
Ubiquitin_evol = Ubiquitin_evol.filter_by([x for x in Ubiquitin_evol['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del Ubiquitin_evol['id']
Ubiquitin_evol = Ubiquitin_evol.add_row_number('id',0)

pred = NoUba1_model.predict(Ubiquitin_evol)
stats.pearsonr(pred, Ubiquitin_evol['scaled_effect1'])

stats.pearsonr(pred, Ubiquitin_evol['scaled_effect1'])
stats.spearmanr(pred, Ubiquitin_evol['scaled_effect1'])

SpearmanrResult(correlation=0.40505687537971141, pvalue=1.6490824380950779e-50)

## Yap65

In [24]:
unpickler = gl._gl_pickle.GLUnpickler(filename = '/net/fowler/vol1/home/vegray/metaDMS/dato/scaled/models/LOPO_models/NoYap65_model_regressionModel_noSELCO_limited_2017-02-08')
NoYap65_model = unpickler.load()
unpickler.close()

In [ ]:
## Missing structural features
yap65_str = data_str.filter_by([x for x in data_str['dms_id'].unique() if x in ['WW_domain']], 'dms_id')
yap65_str = yap65_str.filter_by([x for x in yap65_str['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del yap65_str['id']
yap65_str = yap65_str.add_row_number('id',0)

pred = NoYap65_model.predict(yap65_str)

stats.pearsonr(pred, yap65_str['scaled_effect1'])
#stats.spearmanr(pred, yap65_str['scaled_effect1'])

In [26]:
## Missing evolutionary features
yap65_evol = data_evol.filter_by([x for x in data_evol['dms_id'].unique() if x in ['WW_domain']], 'dms_id')
yap65_evol = yap65_evol.filter_by([x for x in yap65_evol['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del yap65_evol['id']
yap65_evol = yap65_evol.add_row_number('id',0)

pred = NoYap65_model.predict(yap65_evol)
stats.pearsonr(pred, yap65_evol['scaled_effect1'])

stats.pearsonr(pred, yap65_evol['scaled_effect1'])
stats.spearmanr(pred, yap65_evol['scaled_effect1'])

SpearmanrResult(correlation=-0.027704038256774081, pvalue=0.59880947272263896)

## Hsp90 

In [27]:
unpickler = gl._gl_pickle.GLUnpickler(filename = '/net/fowler/vol1/home/vegray/metaDMS/dato/scaled/models/LOPO_models/Nohsp90_model_regressionModel_noSELCO_limited_2017-02-08')
Nohsp90_model = unpickler.load()
unpickler.close()

In [ ]:
## Missing structural features
hsp90_str = data_str.filter_by([x for x in data_str['dms_id'].unique() if x in ['hsp90']], 'dms_id')
hsp90_str = hsp90_str.filter_by([x for x in hsp90_str['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del hsp90_str['id']
hsp90_str = hsp90_str.add_row_number('id',0)

pred = Nohsp90_model.predict(hsp90_str)

stats.pearsonr(pred, hsp90_str['scaled_effect1'])
#stats.spearmanr(pred, hsp90_str['scaled_effect1'])

In [29]:
## Missing evolutionary features
hsp90_evol = data_evol.filter_by([x for x in data_evol['dms_id'].unique() if x in ['hsp90']], 'dms_id')
hsp90_evol = hsp90_evol.filter_by([x for x in hsp90_evol['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del hsp90_evol['id']
hsp90_evol = hsp90_evol.add_row_number('id',0)

pred = Nohsp90_model.predict(hsp90_evol)
stats.pearsonr(pred, hsp90_evol['scaled_effect1'])

stats.pearsonr(pred, hsp90_evol['scaled_effect1'])
#stats.spearmanr(pred, hsp90_evol['scaled_effect1'])

(0.26119716418033989, 1.0302937476021656e-63)

## Protein G

In [30]:
unpickler = gl._gl_pickle.GLUnpickler(filename = '/net/fowler/vol1/home/vegray/metaDMS/dato/scaled/models/LOPO_models/Nogb1_model_regressionModel_noSELCO_limited_2017-02-08')
Nogb1_model = unpickler.load()
unpickler.close()

In [ ]:
## Missing structural features
gb1_str = data_str.filter_by([x for x in data_str['dms_id'].unique() if x in ['gb1']], 'dms_id')
gb1_str = gb1_str.filter_by([x for x in gb1_str['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del gb1_str['id']
gb1_str = gb1_str.add_row_number('id',0)

pred = Nogb1_model.predict(gb1_str)

stats.pearsonr(pred, gb1_str['scaled_effect1'])
#stats.spearmanr(pred, gb1_str['scaled_effect1'])

In [32]:
## Missing evolutionary features
gb1_evol = data_evol.filter_by([x for x in data_evol['dms_id'].unique() if x in ['gb1']], 'dms_id')
gb1_evol = gb1_evol.filter_by([x for x in gb1_evol['mut_type'].unique() if x not in ['synonymous']], 'mut_type')
del gb1_evol['id']
gb1_evol = gb1_evol.add_row_number('id',0)

pred = Nogb1_model.predict(gb1_evol)
stats.pearsonr(pred, gb1_evol['scaled_effect1'])

stats.pearsonr(pred, gb1_evol['scaled_effect1'])
#stats.spearmanr(pred, gb1_evol['scaled_effect1'])

(0.38433112371687506, 4.0442515474933188e-38)